<a href="https://colab.research.google.com/github/tonyjan02/soccerdb/blob/main/pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### CIS 550 Pre-Processing

# Set-Up: Imports with Kaggle 

In [ ]:
# importing necessary libraries
import json
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from google.colab import drive
from sklearn.model_selection import train_test_split

In [ ]:
!apt update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 340 kB in 3s (119 kB/s)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
23 packages can be upgraded. Run 'apt list --upgradable' to see 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d davidcariboo/player-scores

player-scores.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/player-scores.zip

Archive:  /content/player-scores.zip
replace appearances.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: appearances.csv         
  inflating: club_games.csv          
  inflating: clubs.csv               
  inflating: competitions.csv        
  inflating: game_events.csv         
  inflating: games.csv               
  inflating: player_valuations.csv   
  inflating: players.csv             


# Cleaning Players Dataset

In [ ]:
players_df = pd.read_csv("players.csv")

In [ ]:
players_df.columns

Index(['player_id', 'name', 'current_club_id', 'current_club_name',
       'country_of_citizenship', 'country_of_birth', 'city_of_birth',
       'date_of_birth', 'position', 'sub_position', 'foot', 'height_in_cm',
       'market_value_in_eur', 'highest_market_value_in_eur', 'agent_name',
       'contract_expiration_date', 'current_club_domestic_competition_id',
       'first_name', 'last_name', 'player_code', 'image_url', 'last_season',
       'url'],
      dtype='object')

In [ ]:
players_df = players_df.drop(columns=["current_club_name", "country_of_birth", "city_of_birth", "sub_position", "agent_name", "contract_expiration_date", "current_club_domestic_competition_id", "url", "player_code", "first_name", "last_name"])

In [ ]:
players_df = players_df.rename(columns={"name": "player_name", "current_club_id": "club_id", "country_of_citizenship": "country"})
len(players_df.index) # pre-drop

27456

In [ ]:
players_df = players_df.dropna(subset=["player_id", "player_name"])
len(players_df.index) # post-drop, no null values for core attributes

27456

In [ ]:
players_df

,player_id,player_name,club_id,country,date_of_birth,position,foot,height_in_cm,market_value_in_eur,highest_market_value_in_eur,image_url,last_season
0,134354,Ian Raeymaekers,498,Belgium,1995-01-30,Attack,Right,0,NaN,50000.0,https://img.a.transfermarkt.technology/portrai...,2012
1,99946,Mohamed Camara,1095,Guinea,1990-09-20,Attack,Right,180,NaN,300000.0,https://img.a.transfermarkt.technology/portrai...,2012
2,76948,Pablo Olivera,979,Uruguay,1987-12-08,Attack,Right,175,25000.0,600000.0,https://img.a.transfermarkt.technology/portrai...,2012
3,108372,Aliosman Aydin,38,Turkey,1992-02-06,Attack,Right,178,NaN,125000.0,https://img.a.transfermarkt.technology/portrai...,2012
4,78820,Jaime Alfonso Ruiz,354,Colombia,1984-01-03,Attack,Right,184,NaN,1700000.0,https://img.a.transfermarkt.technology/portrai...,2012
...,...,...,...,...,...,...,...,...,...,...,...,...
27451,178313,Jean-Eudes Aholou,667,Cote d'Ivoire,1994-03-20,Midfield,Left,186,3000000.0,10000000.0,https://img.a.transfermarkt.technology/portrai...,2021
27452,250844,Alexander Djiku,667,Ghana,1994-08-09,Defender,Right,182,8000000.0,10000000.0,https://img.a.transfermarkt.technology/portrai...,2021
27453,453016,Anthony Caci,667,France,1997-07-01,Defender,Right,184,7000000.0,7000000.0,https://img.a.transfermarkt.technology/portrai...,2021
27454,880330,Moïse Sahi Dion,667,Cote d'Ivoire,2001-12-20,Attack,Left,175,1200000.0,1200000.0,https://img.a.transfermarkt.technology/portrai...,2021


# Cleaning Clubs Dataset

In [ ]:
clubs_df = pd.read_csv("clubs.csv")

In [ ]:
clubs_df.columns

Index(['club_id', 'club_code', 'name', 'domestic_competition_id',
       'total_market_value', 'squad_size', 'average_age', 'foreigners_number',
       'foreigners_percentage', 'national_team_players', 'stadium_name',
       'stadium_seats', 'net_transfer_record', 'coach_name', 'url'],
      dtype='object')

In [ ]:
clubs_df = clubs_df.drop(columns=["club_code", "domestic_competition_id", "foreigners_number", "foreigners_percentage", "national_team_players", "stadium_name", "stadium_seats", "net_transfer_record", "url"])

In [ ]:
clubs_df = clubs_df.rename(columns={"name": "club_name"})
len(clubs_df.index) # pre-drop

401

In [ ]:
clubs_df = clubs_df.dropna(subset=["club_id", "club_name"])
len(clubs_df.index) # post-drop, no null values for core attributes

401

In [ ]:
clubs_df

,club_id,club_name,total_market_value,squad_size,average_age,coach_name
0,1032,Fc Reading,33.66,26,25.9,Brian McDermott
1,2323,Orduspor,NaN,0,NaN,Héctor Cúper
2,1387,Acn Siena 1904,4.32,30,26.2,Serse Cosmi
3,3592,Kryvbas Kryvyi Rig,2.95,24,24.5,Oleg Taran
4,1071,Wigan Athletic,12.38,29,26.5,Roberto Martínez
...,...,...,...,...,...,...
396,969,Montpellier Hsc,99.86,26,25.3,Olivier Dall'Oglio
397,1041,Olympique Lyon,298.04,28,24.6,Peter Bosz
398,1421,Stade Reims,118.40,30,23.9,Óscar García
399,417,Ogc Nizza,221.63,26,24.5,Christophe Galtier


# Cleaning Games Dataset

In [ ]:
games_df = pd.read_csv("games.csv")

In [ ]:
games_df.columns

Index(['game_id', 'competition_id', 'competition_type', 'season', 'round',
       'date', 'home_club_id', 'away_club_id', 'home_club_goals',
       'away_club_goals', 'aggregate', 'home_club_position',
       'away_club_position', 'club_home_name', 'club_away_name',
       'home_club_manager_name', 'away_club_manager_name', 'stadium',
       'attendance', 'referee', 'url'],
      dtype='object')

In [ ]:
games_df

,game_id,competition_id,competition_type,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,...,home_club_position,away_club_position,club_home_name,club_away_name,home_club_manager_name,away_club_manager_name,stadium,attendance,referee,url
0,2229332,DFL,other,2012,Final,2012-08-12,27,16,2,1,...,-1,-1,Fc Bayern Munchen,Borussia Dortmund,Jupp Heynckes,Jürgen Klopp,Allianz Arena,69000,Michael Weiner,https://www.transfermarkt.co.uk/spielbericht/i...
1,2244388,SUC,other,2012,final 1st leg,2012-08-22,131,418,3,2,...,-1,-1,Fc Barcelona,Real Madrid,Tito Vilanova,José Mourinho,Spotify Camp Nou,91728,Carlos Clos Gómez,https://www.transfermarkt.co.uk/spielbericht/i...
2,2269557,CDR,domestic_cup,2012,4th round 2nd leg,2012-11-28,3709,4032,0,0,...,-1,-1,Fc Getafe,NaN,Luis García,Claudio Barragán,Coliseum Alfonso Pérez,2000,Pedro Jesús Pérez Montero,https://www.transfermarkt.co.uk/spielbericht/i...
3,2254432,CDR,domestic_cup,2012,First Round Replay,2012-08-30,21322,7077,1,0,...,-1,-1,NaN,NaN,Pedro Buenaventura,Pato,El Palmar,0,Andrés Manuel Ceballos Silva,https://www.transfermarkt.co.uk/spielbericht/i...
4,2221759,DFB,domestic_cup,2012,First Round,2012-08-20,109,27,0,4,...,-1,-1,NaN,Fc Bayern Munchen,Oscar Corrochano,Jupp Heynckes,Jahnstadion,12500,Guido Winkmann,https://www.transfermarkt.co.uk/spielbericht/i...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61192,3928809,GRP,domestic_cup,2022,Third Round,2022-10-05,21600,4585,0,1,...,-1,-1,NaN,NaN,Georgios Amanatidis,Periklis Amanatidis,Gipedo Neas Efkarpias,0,Dimitrios Takidis,https://www.transfermarkt.co.uk/spielbericht/i...
61193,3942778,GRP,domestic_cup,2022,Fifth Round,2022-10-19,3062,2079,1,0,...,-1,-1,NaN,Veria Nps,Leonidas Vokolos,Konstantinos Anyfantakis,Grigoris Lambrakis Stadium,0,Stavros Tsimenteridis,https://www.transfermarkt.co.uk/spielbericht/i...
61194,3962963,GRP,domestic_cup,2022,last 16 2nd leg,2023-01-10,1091,9,2,0,...,-1,-1,Paok Thessaloniki,NaN,Razvan Lucescu,Nikolaos Anastopoulos,Toumba Stadium,0,Alexandros Katsikogiannis,https://www.transfermarkt.co.uk/spielbericht/i...
61195,3962951,GRP,domestic_cup,2022,last 16 2nd leg,2023-01-11,3060,683,2,2,...,-1,-1,Atromitos Athen,Olympiakos Piraus,Chris Coleman,Michel,Stadio Peristeriou,0,Georgios Tzovaras,https://www.transfermarkt.co.uk/spielbericht/i...


In [ ]:
games_df = games_df.drop(columns=["competition_type", "home_club_position", "away_club_position", "club_home_name", "club_away_name",
       "home_club_manager_name", "away_club_manager_name", "referee", "url", "aggregate"])
len(games_df) # pre-drop

61197

In [ ]:
games_df = games_df.dropna(subset=["game_id", "home_club_id", "away_club_id", "date", "round"])
games_df = games_df.rename(columns={"date": "game_date"})
len(games_df) # post-drop, no null values for core attributes

61197

In [ ]:
games_df

,game_id,competition_id,season,round,game_date,home_club_id,away_club_id,home_club_goals,away_club_goals,stadium,attendance
0,2229332,DFL,2012,Final,2012-08-12,27,16,2,1,Allianz Arena,69000
1,2244388,SUC,2012,final 1st leg,2012-08-22,131,418,3,2,Spotify Camp Nou,91728
2,2269557,CDR,2012,4th round 2nd leg,2012-11-28,3709,4032,0,0,Coliseum Alfonso Pérez,2000
3,2254432,CDR,2012,First Round Replay,2012-08-30,21322,7077,1,0,El Palmar,0
4,2221759,DFB,2012,First Round,2012-08-20,109,27,0,4,Jahnstadion,12500
...,...,...,...,...,...,...,...,...,...,...,...
61192,3928809,GRP,2022,Third Round,2022-10-05,21600,4585,0,1,Gipedo Neas Efkarpias,0
61193,3942778,GRP,2022,Fifth Round,2022-10-19,3062,2079,1,0,Grigoris Lambrakis Stadium,0
61194,3962963,GRP,2022,last 16 2nd leg,2023-01-10,1091,9,2,0,Toumba Stadium,0
61195,3962951,GRP,2022,last 16 2nd leg,2023-01-11,3060,683,2,2,Stadio Peristeriou,0


# Cleaning Appearances Dataset

In [ ]:
appearances_df = pd.read_csv("appearances.csv")

In [ ]:
appearances_df.columns

Index(['appearance_id', 'game_id', 'player_id', 'player_club_id',
       'player_current_club_id', 'date', 'player_name', 'competition_id',
       'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played'],
      dtype='object')

In [ ]:
appearances_df

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
0,2483937_52453,2483937,52453,28095,28095,2014-08-08,Haris Handzic,RU1,0,0,0,0,90
1,2479929_67064,2479929,67064,28095,4128,2014-08-03,Felicio Brown Forbes,RU1,0,0,0,0,90
2,2483937_67064,2483937,67064,28095,4128,2014-08-08,Felicio Brown Forbes,RU1,0,0,0,0,90
3,2484582_67064,2484582,67064,28095,4128,2014-08-13,Felicio Brown Forbes,RU1,0,0,0,0,55
4,2485965_67064,2485965,67064,28095,4128,2014-08-16,Felicio Brown Forbes,RU1,0,0,0,0,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049580,3589536_555076,3589536,555076,1041,3911,2022-04-20,Romain Faivre,FR1,0,0,0,0,90
1049581,3589565_555076,3589565,555076,1041,3911,2022-05-01,Romain Faivre,FR1,0,0,0,0,7
1049582,3589576_555076,3589576,555076,1041,3911,2022-05-08,Romain Faivre,FR1,0,0,0,0,45
1049583,3589582_555076,3589582,555076,1041,3911,2022-05-14,Romain Faivre,FR1,0,0,0,0,64


In [ ]:
appearances_df = appearances_df.drop(columns=["date", "appearance_id", "player_name", "player_current_club_id", "competition_id"])
len(appearances_df) # pre-drop

1049585

In [ ]:
appearances_df = appearances_df.dropna(subset=["game_id", "player_id", "player_club_id"])
len(appearances_df) # post-drop, some null values for core attributes

1049585

In [ ]:
appearances_df

,game_id,player_id,player_club_id,yellow_cards,red_cards,goals,assists,minutes_played
0,2483937,52453,28095,0,0,0,0,90
1,2479929,67064,28095,0,0,0,0,90
2,2483937,67064,28095,0,0,0,0,90
3,2484582,67064,28095,0,0,0,0,55
4,2485965,67064,28095,0,0,0,0,90
...,...,...,...,...,...,...,...,...
1049580,3589536,555076,1041,0,0,0,0,90
1049581,3589565,555076,1041,0,0,0,0,7
1049582,3589576,555076,1041,0,0,0,0,45
1049583,3589582,555076,1041,0,0,0,0,64


# Export Tables for DB

In [ ]:
# export tables
players_df.to_csv('players_cleaned.csv')
clubs_df.to_csv('clubs_cleaned.csv')
games_df.to_csv('games_cleaned.csv')
appearances_df.to_csv('appearances_cleaned.csv')